<a href="https://colab.research.google.com/github/Alexiagmz/Regresi-n-Logistica/blob/main/M3_Actividad_4_(Regresi%C3%B3n_Log%C3%ADstica)_Barcelona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Cargamos librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
#Carga desde un archivo .csv sin indice y eliminar filas innecesarias
barcelona=pd.read_csv("Barcelona_limpios_valores.csv")
barcelona.head(5)

,Unnamed: 0,id,host_id,host_total_listings_count,accommodates,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,...,host_has_profile_pic,host_identity_verified,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,bathrooms,bedrooms,amenities,calendar_last_scraped
0,0,269467,1411775,7.0,2.0,176.0,7.0,760.0,7.0,7.0,...,t,t,la Sagrada Família,Eixample,Entire loft,Entire home/apt,1.0,1.0,"[""Kitchen"", ""Patio or balcony"", ""Host greets y...",2024-06-15
1,1,270197,1414702,112.0,6.0,74.0,32.0,760.0,32.0,32.0,...,t,t,el Raval,Ciutat Vella,Entire rental unit,Entire home/apt,1.5,4.0,"[""Kitchen"", ""Washer"", ""Smoking allowed"", ""Wifi...",2024-06-15
2,2,18674,71615,45.0,3.0,257.0,1.0,1125.0,1.0,4.0,...,t,t,la Sagrada Família,Eixample,Entire rental unit,Entire home/apt,2.0,3.0,"[""Pack \u2019n play/Travel crib"", ""Free street...",2024-06-15
3,3,272282,1425070,2.0,3.0,136.0,4.0,1125.0,3.0,3.0,...,t,t,l'Antiga Esquerra de l'Eixample,Eixample,Entire rental unit,Entire home/apt,Unknown,3.0,"[""Pack \u2019n play/Travel crib"", ""Kitchen"", ""...",2024-06-15
4,4,23197,90417,13.0,5.0,249.0,3.0,32.0,1.0,7.0,...,t,t,el Besòs i el Maresme,Sant Martí,Entire rental unit,Entire home/apt,2.0,3.0,"[""Free street parking"", ""Neutral body soap"", ""...",2024-06-15


In [3]:
# Reemplazar 'unknown' por NaN
barcelona['bathrooms'] = barcelona['bathrooms'].replace('Unknown', np.nan)
barcelona['bedrooms'] = barcelona['bedrooms'].replace('Unknown', np.nan)
# convertir a float
barcelona['bathrooms'] = barcelona['bathrooms'].astype(float)
barcelona['bedrooms'] = barcelona['bedrooms'].astype(float)

In [4]:
#Rellenamos valores nulos
barcelona= barcelona.fillna(method='bfill')
barcelona= barcelona.fillna(method='ffill')

<ipython-input-4-c897af8e098f>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='bfill')
<ipython-input-4-c897af8e098f>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='ffill')


In [5]:
barcelona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18923 entries, 0 to 18922
Data columns (total 45 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    18923 non-null  int64  
 1   id                            18923 non-null  int64  
 2   host_id                       18923 non-null  int64  
 3   host_total_listings_count     18923 non-null  float64
 4   accommodates                  18923 non-null  float64
 5   price                         18923 non-null  float64
 6   minimum_nights                18923 non-null  float64
 7   maximum_nights                18923 non-null  float64
 8   minimum_minimum_nights        18923 non-null  float64
 9   maximum_minimum_nights        18923 non-null  float64
 10  minimum_maximum_nights        18923 non-null  float64
 11  maximum_maximum_nights        18923 non-null  float64
 12  minimum_nights_avg_ntm        18923 non-null  float64
 13  m

### **Regresión logística**

**1. **

Creación de Categorias

In [6]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=barcelona['price'].max()
Min=barcelona['price'].min()
Limites= [Min, Max]
Limites

[10.0, 414.0]

In [10]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos=np.linspace(10.0, 414.0, 3)
intervalos

array([ 10., 212., 414.])

In [11]:
#Creamos las categorías
categorias= ["barato", "caro"]

In [12]:
#Finalmente creamos las categorías en la columna numérica
barcelona['price']=pd.cut(x= barcelona['price'], bins=intervalos, labels= categorias)

In [13]:
#Rellenamos valores nulos
barcelona= barcelona.fillna(method='bfill')
barcelonaf= barcelona.fillna(method='ffill')

<ipython-input-13-2a919a1b957c>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='bfill')
<ipython-input-13-2a919a1b957c>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelonaf= barcelona.fillna(method='ffill')


In [14]:
barcelona['price'].isnull().sum()

0

Correlación

In [15]:
vars_Indep = barcelona[['bedrooms']]
vars_Dep = barcelona['price']

In [16]:
#Redefinimos las variables
x= vars_Indep
y= vars_Dep

In [17]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test, X_train, Y_test, Y_train= train_test_split(x,y, test_size=0.3, random_state=None)

In [18]:
escalar = StandardScaler()

In [19]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [20]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [21]:
#Entrenamos el modelo
algoritmo.fit(X_train, Y_train)

LogisticRegression()

In [22]:
#realizamos una prediccion
y_pred = algoritmo.predict(X_test)
y_pred

array(['barato', 'barato', 'barato', ..., 'barato', 'barato', 'barato'],
      dtype=object)

In [23]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test, y_pred)
print('Matriz de Confusion')
print(matriz)

Matriz de Confusion
[[10646    32]
 [ 2567     1]]


In [24]:
#CAlculamos la prediccion del modelo
from sklearn.metrics import precision_score

precision = precision_score(Y_test, y_pred, average="binary", pos_label="barato")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.8057216377809733


In [25]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(Y_test, y_pred)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.803789823342896


In [27]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(Y_test, y_pred, average="binary", pos_label="barato")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
0.9970031841168758


**2. **

Creacion de categoria

In [133]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=barcelona['review_scores_cleanliness'].max()
Min=barcelona['review_scores_cleanliness'].min()
Limites= [Min, Max]
Limites

['sucio', 'limpio']

In [134]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos1=np.linspace(0.0, 5.0,3)
intervalos1

array([0. , 2.5, 5. ])

In [135]:
#Creamos las categorías
categorias1= ["sucio", "limpio"]

In [136]:
barcelona['review_scores_cleanliness']

,review_scores_cleanliness
0,limpio
1,limpio
2,limpio
3,limpio
4,limpio
...,...
18918,limpio
18919,limpio
18920,limpio
18921,limpio


In [137]:
#Finalmente creamos las categorías en la columna numérica
barcelona['review_scores_cleanliness']=pd.cut(x= barcelona['review_scores_cleanliness'], bins=intervalos1, labels= categorias1 )

TypeError: '<' not supported between instances of 'float' and 'str'

In [138]:
#Rellenamos valores nulos
barcelona= barcelona.fillna(method='bfill')
barcelona= barcelona.fillna(method='ffill')

<ipython-input-138-c897af8e098f>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='bfill')
<ipython-input-138-c897af8e098f>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='ffill')


In [139]:
barcelona['review_scores_cleanliness'].isnull().sum()

0

In [140]:
vars_Indep_1 = barcelona[['bedrooms']]
vars_Dep_1 = barcelona['review_scores_cleanliness']

In [141]:
#Redefinimos las variables
x= vars_Indep_1
y= vars_Dep_1

In [142]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test1, X_train1, Y_test1, Y_train1= train_test_split(x,y, test_size=0.3, random_state=None)

In [143]:
escalar1 = StandardScaler()

In [144]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train1 = escalar.fit_transform(X_train1)
X_test1 = escalar.transform(X_test1)

In [145]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo1 = LogisticRegression()

In [146]:
#Entrenamos el modelo
algoritmo1.fit(X_train1, Y_train1)

LogisticRegression()

In [147]:
#realizamos una prediccion
y_pred1 = algoritmo1.predict(X_test1)
y_pred1

array(['limpio', 'limpio', 'limpio', ..., 'limpio', 'limpio', 'limpio'],
      dtype=object)

In [148]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test1, y_pred1)
print('Matriz de Confusion')
print(matriz)

Matriz de Confusion
[[13102     0]
 [  144     0]]


In [151]:
#Calculamos la prediccion del modelo
from sklearn.metrics import precision_score

precision = precision_score(Y_test1, y_pred1, average="binary", pos_label="limpio")
print('Precision del modelo:')
print(precision)

Precision del modelo:
0.9891287935980674


In [152]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(Y_test1, y_pred1)
print('Exactitud del modelo:')
print(exactitud)

Exactitud del modelo:
0.9891287935980674


In [153]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(Y_test1, y_pred1, average="binary", pos_label="limpio")
print('Sensibilidad del modelo:')
print(sensibilidad)

Sensibilidad del modelo:
1.0


**3. **

In [52]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=barcelona['accommodates'].max()
Min=barcelona['accommodates'].min()
Limites= [Min, Max]
Limites

[1.0, 7.0]

In [53]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos2=np.linspace(1.0,7.0,3)
intervalos2


array([1., 4., 7.])

In [54]:
#Creamos las categorías
categorias2= ["incomodo", "comodo"]

In [55]:
barcelona['accommodates']

,accommodates
0,2.0
1,6.0
2,3.0
3,3.0
4,5.0
...,...
18918,2.0
18919,4.0
18920,4.0
18921,2.0


In [56]:
#Finalmente creamos las categorías en la columna numérica
barcelona['accommodates']=pd.cut(x= barcelona['accommodates'], bins=intervalos2, labels= categorias2 )


In [57]:
#Rellenamos valores nulos
barcelona= barcelona.fillna(method='bfill')
barcelonaf= barcelona.fillna(method='ffill')


<ipython-input-57-2a919a1b957c>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='bfill')
<ipython-input-57-2a919a1b957c>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelonaf= barcelona.fillna(method='ffill')


In [58]:
barcelona['accommodates'].isnull().sum()

0

In [59]:
vars_Indep_2 = barcelona[['bedrooms']]

vars_Dep_2 = barcelona['accommodates']


In [60]:
#Redefinimos las variables
x= vars_Indep_2
y= vars_Dep_2


In [61]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test2, X_train2, Y_test2, Y_train2= train_test_split(x,y, test_size=0.3, random_state=None)


In [62]:
escalar2 = StandardScaler()

In [63]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train2 = escalar.fit_transform(X_train2)
X_test2 = escalar.transform(X_test2)


In [64]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo2 = LogisticRegression()


In [65]:
#Entrenamos el modelo
algoritmo2.fit(X_train2, Y_train2)


LogisticRegression()

In [66]:
#realizamos una prediccion
y_pred2 = algoritmo2.predict(X_test2)
y_pred2


array(['comodo', 'incomodo', 'incomodo', ..., 'incomodo', 'incomodo',
       'incomodo'], dtype=object)

In [67]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test2, y_pred2)
print('Matriz de Confusion')
print(matriz)


Matriz de Confusion
[[ 306 2431]
 [ 621 9888]]


In [68]:
#CAlculamos la prediccion del modelo
from sklearn.metrics import precision_score
precision = precision_score(Y_test2, y_pred2, average="binary", pos_label="incomodo")
print('Precision del modelo:')
print(precision)


Precision del modelo:
0.8026625537787158


In [69]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(Y_test2, y_pred2)
print('Exactitud del modelo:')
print(exactitud)


Exactitud del modelo:
0.7695908198701494


In [70]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(Y_test2, y_pred2, average="binary", pos_label="incomodo")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.9409077933200114


**4. **

In [71]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=barcelona['review_scores_rating'].max()
Min=barcelona['review_scores_rating'].min()
Limites= [Min, Max]
Limites


[0.0, 5.0]

In [72]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos3=np.linspace(0.0,5.0, 3)
intervalos3


array([0. , 2.5, 5. ])

In [73]:
#Creamos las categorías
categorias3= ["muy mala", "muy buena"]


In [74]:
barcelona['review_scores_rating']

,review_scores_rating
0,4.74
1,4.00
2,4.35
3,4.92
4,4.79
...,...
18918,0.00
18919,0.00
18920,0.00
18921,0.00


In [75]:
#Finalmente creamos las categorías en la columna numérica
barcelona['review_scores_rating']=pd.cut(x= barcelona['review_scores_rating'], bins=intervalos3, labels= categorias3 )


In [76]:
#Rellenamos valores nulos
barcelona= barcelona.fillna(method='bfill')
barcelona= barcelona.fillna(method='ffill')


<ipython-input-76-c897af8e098f>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='bfill')
<ipython-input-76-c897af8e098f>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='ffill')


In [77]:
barcelona['review_scores_rating'].isnull().sum()

0

In [78]:
vars_Indep_3 = barcelona[['bedrooms']]

vars_Dep_3 = barcelona['review_scores_rating']


In [79]:
#Redefinimos las variables
x= vars_Indep_3
y= vars_Dep_3


In [80]:

#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test3, X_train3, Y_test3, Y_train3= train_test_split(x,y, test_size=0.3, random_state=None)


In [81]:
escalar3 = StandardScaler()

In [82]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train3 = escalar.fit_transform(X_train3)
X_test3 = escalar.transform(X_test3)


In [83]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo3 = LogisticRegression()


In [84]:
#Entrenamos el modelo
algoritmo3.fit(X_train3, Y_train3)


LogisticRegression()

In [85]:
#realizamos una prediccion
y_pred3 = algoritmo3.predict(X_test3)
y_pred3


array(['muy buena', 'muy buena', 'muy buena', ..., 'muy buena',
       'muy buena', 'muy buena'], dtype=object)

In [86]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test3, y_pred3)
print('Matriz de Confusion')
print(matriz)


Matriz de Confusion
[[13080     0]
 [  166     0]]


In [87]:
#CAlculamos la prediccion del modelo
from sklearn.metrics import precision_score
precision = precision_score(Y_test3, y_pred3, average="binary", pos_label="muy mala")
print('Precision del modelo:')
print(precision)


Precision del modelo:
0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [88]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(Y_test3, y_pred3)
print('Exactitud del modelo:')
print(exactitud)


Exactitud del modelo:
0.9874679148422165


In [89]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(Y_test3, y_pred3, average="binary", pos_label="muy mala")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.0


**5. **

In [90]:
#Obtenemos el limite superior y el límite inferior de la columna objetivo
Max=barcelona['number_of_reviews'].max()
Min=barcelona['number_of_reviews'].min()
Limites= [Min, Max]
Limites


[0.0, 120.0]

In [91]:
barcelona['number_of_reviews']

,number_of_reviews
0,90.0
1,2.0
2,41.0
3,18.1
4,78.0
...,...
18918,0.0
18919,0.0
18920,0.0
18921,0.0


In [92]:
#Categorización de variables
#Declaramos 2 intervalos
intervalos4=np.linspace(0.0, 120.0, 3)
intervalos4


array([  0.,  60., 120.])

In [93]:
#Creamos las categorías
categorias4= ["pocas", "muchas"]


In [94]:
barcelona['number_of_reviews']

,number_of_reviews
0,90.0
1,2.0
2,41.0
3,18.1
4,78.0
...,...
18918,0.0
18919,0.0
18920,0.0
18921,0.0


In [95]:
#Finalmente creamos las categorías en la columna numérica
barcelona['number_of_reviews']=pd.cut(x= barcelona['number_of_reviews'], bins=intervalos4, labels= categorias4 )


In [98]:
#Rellenamos valores nulos
barcelona= barcelona.fillna(method='bfill')
barcelona= barcelona.fillna(method='ffill')


<ipython-input-98-c897af8e098f>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='bfill')
<ipython-input-98-c897af8e098f>:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  barcelona= barcelona.fillna(method='ffill')


In [99]:
barcelona['number_of_reviews'].isnull().sum()

0

In [100]:
vars_Indep_4 = barcelona[['bedrooms']]

vars_Dep_4 = barcelona['number_of_reviews']


In [101]:
#Redefinimos las variables
x= vars_Indep_4
y= vars_Dep_4


In [102]:
#dividimos el conjunto de datos en la parte de entrenamiento y pruba
X_test4, X_train4, Y_test4, Y_train4= train_test_split(x,y, test_size=0.3, random_state=None)


In [103]:
escalar4 = StandardScaler()

In [104]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, Utilizaremos
X_train4 = escalar.fit_transform(X_train4)
X_test4 = escalar.transform(X_test4)


In [105]:
#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo4 = LogisticRegression()


In [106]:
#Entrenamos el modelo
algoritmo4.fit(X_train4, Y_train4)


LogisticRegression()

In [107]:
#realizamos una prediccion
y_pred4 = algoritmo4.predict(X_test4)
y_pred4


array(['pocas', 'pocas', 'pocas', ..., 'pocas', 'pocas', 'pocas'],
      dtype=object)

In [108]:
#Verifico la matriz de confusion
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(Y_test4, y_pred4)
print('Matriz de Confusion')
print(matriz)


Matriz de Confusion
[[    0  1523]
 [    3 11720]]


In [110]:
#Calculamos la prediccion del modelo
from sklearn.metrics import precision_score
precision = precision_score(Y_test4, y_pred4, average="binary", pos_label="pocas")
print('Precision del modelo:')
print(precision)


Precision del modelo:
0.8849958468624934


In [111]:
#Calculamos la exactitud del modelo
from sklearn.metrics import accuracy_score
exactitud = accuracy_score(Y_test4, y_pred4)
print('Exactitud del modelo:')
print(exactitud)


Exactitud del modelo:
0.8847954099350748


In [112]:
#calculamos la sensibilidad del modelo
from sklearn.metrics import recall_score
sensibilidad = recall_score(Y_test4, y_pred4, average="binary", pos_label="pocas")
print('Sensibilidad del modelo:')
print(sensibilidad)


Sensibilidad del modelo:
0.9997440928090079
